#afw.Image.ExposureF('file.fits.fz[i]') returns the image in 'file.fits.fz[1]'

* [DM-2599](https://jira.lsstcorp.org/browse/DM-2599), John Swinbank, 3 SPs.

Issue description:

> It seems that afwImage.ExposureF ignores the extension number when this is passed on as part of the filename and uses the image in extension number 1. This is not the case with afwImage.MaskedImageF which correctly uses the input extension number passed in the same way.
> The problem has been checked on OSX Yosemite 10.10.3 with 
> the is illustrated in the following code https://gist.github.com/anonymous/d10c4a79d94c1393a493
> which also requires the following image in the working directory:
> http://www.astro.washington.edu/users/krughoff/data/c4d_130830_040651_ooi_g_d1.fits.fz
 
This notebook has been checked to run against the release tagged `w_2015_26`.

The issue here is broader than the description above leads one to suppose. In fact, this is a confusing result of our attempt to shoe-horn a rich data model into the FITS format.

Both masked images (`lsst.afw.image.MaskedImageF`, etc) and exposures (`lsst.afw.image.ExposureF`, etc) provide an image with associated mask and variance planes. When serialized to FITS, they are stored as successive extensions in a multi-extension FITS file. See [LSST Doxygen](https://lsst-web.ncsa.illinois.edu/doxygen/x_masterDoxyDoc/afw_sec_image_i_o.html) for details. 

For the purposes of this demo, we're going to want to look at log output from the LSST stack. Unfortunately, there's not an obvious way to cleanly include this in an IPython notebook (suggestions welcome...). As a workaround, we'll define a context manager as follows:

In [16]:
from __future__ import print_function
from contextlib import contextmanager
from tempfile import NamedTemporaryFile
import lsst.pex.logging as logging

@contextmanager
def capture_log():
    tf = NamedTemporaryFile()
    root = logging.getDefaultLog()
    root.addDestination(logging.FileDestination(tf.name, logging.BriefFormatter()))
    yield
    print(tf.read())

In [7]:
import lsst.afw.image as afwImage

In [17]:
with capture_log():
    im = afwImage.MaskedImageF("c4d_130830_040651_ooi_g_d1.fits.fz[2]")

afw.image.MaskedImage WARNING: Expected extension type not found: IMAGE

